In [1]:
import numpy as np
from scipy.stats import norm
import mean_confidence_interval as conf
import geometric_brownian_motion as gbm

In [2]:
class AsianOption(object):
    '''
    Class for Asian Option valuation
    contract - option contract (put or call)
    S0 - initial stock value
    K - strike price
    T - time to maturity (years)
    r - annual risk free rate 
    mu - expected return
    sigma - volatility
    steps - number of steps in discretization
    reps - number of simulations 
    '''
    # Constructor
    def __init__(self,contract,S0,K,T,r,mu,sigma,steps,reps):
        self.contract = contract
        self.S0 = float(S0)
        self.K = float(K)
        self.T = float(T)
        self.r = float(r)
        self.mu = float(mu)
        self.sigma = float(sigma)
        self.steps = steps
        self.reps = reps
        self.interval = float(T/steps)
        if (contract != 'call') & (contract != 'put'):
            raise ValueError('Invalid Contract Type. Specify <call> or <put>')
        self.value = self.Sim_value()
    def Sim_value(self):
        '''
        Return European option value using Brownian Random Walk Monte-Carlo simulation
        '''
        prices = gbm.BRW(self.mu,self.sigma,self.S0,self.T,self.reps,self.steps)
        ave_price = prices.mean(axis=1)
        values=[]
        for val in ave_price:
            if self.contract =='call':
                values.append(np.exp(-self.r*self.T)*np.maximum(0.0,val - self.K))
            elif self.contract =='put':
                values.append(np.exp(-self.r*self.T)*np.maximum(0.0,self.K - val))
        value, CI_95 = conf.CI(values)
        return value, CI_95

In [3]:
test_call = AsianOption(contract='call',S0=100,K=105,T=1,r=0.05,mu=0.05,sigma=0.2,steps=100,reps=10000)
print 'Simulated Value:',test_call.value

Simulated Value: (3.4653468312307187, 0.12505979776305728)


In [4]:
test_put = AsianOption('put',100,105,1,0.05,0.05,0.2,100,10000)
print 'Simulated Value:',test_put.value

Simulated Value: (5.8881161644685491, 0.1358725630795389)
